In [ ]:
import os
import matplotlib.pyplot as plt
from scipy import misc
from scipy.ndimage import rotate
from PIL import Image
import pandas as pd
import numpy as np
import cv2
#convert train jpg to png and save to the new directory
os.makedirs('/train_pngs')
for filename in os.listdir('baseline_NTA/data/train/images/'):
    image = Image.open('baseline_NTA/data/train/images/'+filename)
    image.save('/train_pngs/'+filename[:-4]+'.png', "PNG")

In [ ]:
#convert test jpg to png and save to the new directory
os.makedirs('/test_pngs')
for filename in os.listdir('baseline_NTA/data/test/images/'):
    image = Image.open('baseline_NTA/data/test/images/'+filename)
    image.save('/test_pngs/'+filename[:-4]+'.png', "PNG")

In [ ]:
#function to rotate images and landmarks
def rot(image, xy, angle):
    im_rot = rotate(image,angle) 
    org_center = (np.array(image.shape[:2][::-1])-1)/2.
    rot_center = (np.array(im_rot.shape[:2][::-1])-1)/2.
    new = []
    xy = xy.values.reshape(194,2)
    for l in xy:
        org = l-org_center
        a = np.deg2rad(angle)
        n = np.array([org[0]*np.cos(a) + org[1]*np.sin(a),
                        -org[0]*np.sin(a) + org[1]*np.cos(a) ])
        new.append(n)
    return im_rot, (new+rot_center).ravel()

In [ ]:
landmarks.loc[2000]=0

In [ ]:
landmarks.tail()

In [ ]:
#rotate imgs and landmarks and save them
landmarks = pd.read_csv('baseline_NTA/data/train/landmarks.csv')
for i in np.arange(landmarks.shape[0]):
    if i<500:
        angle = 15
        name = '_rotated_plus15'
    elif i>501&i<1000:
        angle = -15
        name = '_rotated_minus15'
    elif i>1001&i<1500:
        angle = 30
        name = '_rotated_plus30'
    elif i>1501&i<2000:
        angle = -30
        name = '_rotated_minus30'
        
    f = landmarks['filename'].iloc[i]
    print(f)
    ldm = landmarks.iloc[i, 1:]
    img = cv2.imread('train_pngs/'+f[:-4]+'.png')
    rotated_img, rotated_ldm = rot(img, ldm, angle)
    im = Image.fromarray(rotated_img)
    im.save('train_pngs/'+f[:-4]+name+'.png', "PNG")
    landmarks.loc[2000+i] = 0
    landmarks.iloc[2000+i,0] = f[:-4]+name+'.png'
    landmarks.iloc[2000+i,1:] = rotated_ldm

In [ ]:
landmarks.to_csv('landmarks_aug_15_30.csv')